# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [122]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint 


# Import API key
from api_keys import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [123]:
#setting file path to read in
file = os.path.join("..", "Output_Data", "cities.csv")

#read in csv
city_weather = pd.read_csv(file, delimiter=',')
city_weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hobart,-42.8794,147.3294,68.00,52,75,12.75,AU,1609892714
1,College Station,30.6280,-96.3344,64.40,52,1,8.05,US,1609892615
2,Kapaa,22.0752,-159.3190,78.80,65,75,13.87,US,1609892716
3,Torbay,47.6666,-52.7314,37.99,100,90,17.22,CA,1609892718
4,Vila Franca do Campo,37.7167,-25.4333,55.40,71,75,13.87,PT,1609892719


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [124]:
gmaps.configure(api_key=gkey)

In [151]:
#set map locations and weight 
locations = city_weather[["Lat", "Lng"]].astype(float)
humidity = city_weather["Humidity"].astype(float)

#generate gmap 
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2)

#add heat layer to gmap 
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [158]:
#narrowing down cities based on max temp, wind speed and cloudiness conditions
ideal_weather = city_weather.loc[(city_weather["Max Temp"] >= 73) & (city_weather["Max Temp"] <= 80) 
                                  & (city_weather["Wind Speed"] < 10) & (city_weather["Cloudiness"] == 0)]
ideal_weather.dropna()
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
129,Ondo,7.1000,4.8333,74.21,95,0,2.13,NG,1609892876
208,Maragogi,-9.0122,-35.2225,77.40,75,0,6.80,BR,1609892974
278,Wukari,7.8500,9.7833,75.02,22,0,3.51,NG,1609893067
306,Bara,10.3744,10.7288,73.72,20,0,9.04,NG,1609893036
329,Bolama,11.5833,-15.4667,75.20,69,0,5.82,GW,1609893130
424,Coruripe,-10.1256,-36.1756,76.23,81,0,7.72,BR,1609893250
464,Itaqui,-29.1253,-56.5531,77.05,71,0,7.02,BR,1609893299
522,Lafia,8.4833,8.5167,74.95,23,0,1.10,NG,1609893371
524,Valdivia,-39.8142,-73.2459,73.40,60,0,9.17,CL,1609893299
527,Eenhana,-17.4667,16.3333,77.00,44,0,4.70,NaN,1609893378


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [159]:
#rename ideal weather df
hotel_df = pd.DataFrame(data=ideal_weather)
hotel_df["Hotel Name"] = ""
hotel_df.reindex()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
129,Ondo,7.1000,4.8333,74.21,95,0,2.13,NG,1609892876,
208,Maragogi,-9.0122,-35.2225,77.40,75,0,6.80,BR,1609892974,
278,Wukari,7.8500,9.7833,75.02,22,0,3.51,NG,1609893067,
306,Bara,10.3744,10.7288,73.72,20,0,9.04,NG,1609893036,
329,Bolama,11.5833,-15.4667,75.20,69,0,5.82,GW,1609893130,
424,Coruripe,-10.1256,-36.1756,76.23,81,0,7.72,BR,1609893250,
464,Itaqui,-29.1253,-56.5531,77.05,71,0,7.02,BR,1609893299,
522,Lafia,8.4833,8.5167,74.95,23,0,1.10,NG,1609893371,
524,Valdivia,-39.8142,-73.2459,73.40,60,0,9.17,CL,1609893299,
527,Eenhana,-17.4667,16.3333,77.00,44,0,4.70,NaN,1609893378,


In [160]:
#set call parameters 
params = {"keyword": "Hotel",
          "radius": 5000,
          "type": "lodging",
          "key": gkey}

#
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params)
    response_json = response.json()
    #print(json.dumps(response_json, indent=4, sort_keys=True))
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response_json["results"][0]["name"]
    
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "NaN"
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
129,Ondo,7.1000,4.8333,74.21,95,0,2.13,NG,1609892876,Labim Hotel
208,Maragogi,-9.0122,-35.2225,77.40,75,0,6.80,BR,1609892974,Pousada Camurim Grande
278,Wukari,7.8500,9.7833,75.02,22,0,3.51,NG,1609893067,Rohi Grand Suites
306,Bara,10.3744,10.7288,73.72,20,0,9.04,NG,1609893036,NaN
329,Bolama,11.5833,-15.4667,75.20,69,0,5.82,GW,1609893130,Hotel do Mercado
424,Coruripe,-10.1256,-36.1756,76.23,81,0,7.72,BR,1609893250,Pousada Paradise - Couripe - Alagoas
464,Itaqui,-29.1253,-56.5531,77.05,71,0,7.02,BR,1609893299,Hotel Contursi
522,Lafia,8.4833,8.5167,74.95,23,0,1.10,NG,1609893371,Taal Conference Hotel
524,Valdivia,-39.8142,-73.2459,73.40,60,0,9.17,CL,1609893299,Hotel Dreams Pedro De Valdivia
527,Eenhana,-17.4667,16.3333,77.00,44,0,4.70,NaN,1609893378,Hotel Litu


In [154]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [155]:
#copied code from heatmap above
locations = city_weather[["Lat", "Lng"]].astype(float)
humidity = city_weather["Humidity"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2)
fig.add_layer(heat_layer)

#Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker)

# Display figure
fig


IndexError: list index out of range